<a href="https://colab.research.google.com/github/volpter/ML_comment_selection/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
import joblib

In [3]:
import pandas as pd
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, plot_precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
main_df = pd.read_csv('labeled.csv', sep =',' )


In [5]:
main_df.toxic = main_df.toxic.apply(lambda x: int(x))

In [16]:
train_X, test_X, train_y, test_y = train_test_split(main_df.comment, main_df.toxic, random_state=42, test_size=600)

In [7]:
stemmer=SnowballStemmer(language='russian')
def preprocess(comment):
    t_com = word_tokenize(comment, language='russian')
    t_com = [t for t in t_com if t not in string.punctuation ]
    t_com = [t for t in t_com if t not in stopwords.words('russian')]
    t_com = [stemmer.stem(i) for i in t_com]
    return (t_com)
        

In [17]:
train_X = train_X.apply(preprocess)
test_X = test_X.apply(preprocess)

In [34]:
def unite(l:list):
  temp = ''
  for i in range(len(l)):
    temp+=l[i]
    temp+=' '
  temp = temp[:-2]
  return (temp)

In [35]:
train_set= train_X.apply(unite)
test_set = test_X.apply(unite)

In [53]:
vectorizer = TfidfVectorizer()
model_pipeline = Pipeline([
                          ('vectorizer', vectorizer),
                          ('model', LogisticRegression(random_state=42))
])

In [52]:
param = [{ 'vectorizer__ngram_range': [(1, 1), (1, 2)],
          'vectorizer__norm': ('l1', 'l2', None),
          'model__solver': ['newton-cg', 'lbfgs', 'sag'],
          'model__penalty': ['none', 'l2']
    
}]
gridsearcher = GridSearchCV(model_pipeline, param, cv=2, verbose=1)

In [ ]:
gridsearcher.fit(train_set, train_y)

In [55]:
gridsearcher.best_params_

{'model__penalty': 'none',
 'model__solver': 'sag',
 'vectorizer__ngram_range': (1, 1),
 'vectorizer__norm': 'l1'}

In [ ]:
model = gridsearcher.best_estimator_
joblib.dump(model, 'model.pkl')

In [57]:
model = joblib.load('model.pkl')


In [58]:
prediction = model.predict(test_set)
print (precision_score(prediction, test_y), recall_score(prediction, test_y))

0.7412935323383084 0.8097826086956522
